In [61]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import re

In [62]:
data = pd.read_csv('results.csv', encoding='UTF8')

In [63]:
data.shape

(1005, 6)

In [64]:
#data description
data.describe()

,Titre du poste,Entreprise,Adresse,Salaire,Decription,date
count,1005,1005,1005,299,1005,1005
unique,694,585,150,146,790,33
top,Business Developer H/F,FABERNOVEL,Paris (75),1 000 € par mois,"Cette Talent Company rassemble Designers, Stra...",Postedil y a 30+ jours
freq,46,27,428,13,22,388


In [65]:
#Data Informations
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Titre du poste  1005 non-null   object
 1   Entreprise      1005 non-null   object
 2   Adresse         1005 non-null   object
 3   Salaire         299 non-null    object
 4   Decription      1005 non-null   object
 5   date            1005 non-null   object
dtypes: object(6)
memory usage: 47.2+ KB


In [66]:
data.nunique()

Titre du poste    694
Entreprise        585
Adresse           150
Salaire           146
Decription        790
date               33
dtype: int64

In [67]:
#Checking missing values

data.isnull().sum()

Titre du poste      0
Entreprise          0
Adresse             0
Salaire           706
Decription          0
date                0
dtype: int64

In [68]:
#Dropping the NULL Values from Column, 'Salaire'

data = data.dropna()
data.shape

(299, 6)

### Texte processing

### Lowerization

In [69]:
data = data.applymap(lambda s: s.lower() if type(s) == str else s)

### Salary processing: Minimum, maximum et mean

salary pay type: par heure, par jour, par semaine, par mois, par an

In [70]:
#Creating columns for each salay pay type
data['Par_heure'] = data['Salaire'].apply(lambda x: 1 if 'par heure' in x.lower() else 0 )

In [71]:
data['Par_jour'] = data['Salaire'].apply(lambda x: 1 if 'par jour' in x.lower() else 0 )

In [72]:
data['Par_semaine'] = data['Salaire'].apply(lambda x: 1 if 'par semaine' in x.lower() else 0 )

In [73]:
data['Par_mois'] = data['Salaire'].apply(lambda x: 1 if 'par mois' in x.lower() else 0 )

In [74]:
data['Par_an'] = data['Salaire'].apply(lambda x: 1 if 'par an' in x.lower() else 0 )

In [75]:
salary_range = data['Salaire'].apply(lambda x:
                                   x.replace('€', '')
                                   .replace('par heure', '')
                                   .replace('par an', '')
                                   .replace('par mois', '')
                                   .replace('par semaine', '')
                                   .replace('par jour', '')
                                   .replace(' ','')).astype(str)

In [76]:
data['min_sal'] = salary_range.apply(lambda x: x.split('-')[0])

In [77]:
#bar graph

data.sort_values('Salaire', inplace=True)

fig = px.histogram(data, x='min_sal')
#fig.show()

In [78]:
data['max_sal'] = salary_range.apply(lambda x: x.split('-')[1] if len(x.split('-'))>1 else x).astype(int)

In [79]:
#bar graph

data.sort_values('Salaire', inplace=True)

fig = px.histogram(data, x='max_sal')
#fig.show()

In [80]:
#df[df['min_sal'] + df['max_sal']]
data['mean_sal'] = (pd.to_numeric(data['min_sal']) + pd.to_numeric(data['max_sal']))/2

In [81]:
#Monthly salary
m1 = data['Salaire'].str.contains('par heure')
m2 = data['Salaire'].str.contains('par jour')
m3 = data['Salaire'].str.contains('par semaine') #par mois is a default value
m5 = data['Salaire'].str.contains('par an')

data['monthly_salary'] = np.select([m1, m2, m3, m5],
                                  [data['mean_sal']*152, #converting hourly to monthly
                                   data['mean_sal']*22,   #converting daily to monthly                               
                                   data['mean_sal']*4,    #weeklyly to monthly
                                   data['mean_sal']/12],   #monthly to monthly
                                   default=data['mean_sal']) #default value
data['monthly_salary'].tail().round()

986    6667.0
446     824.0
437    7500.0
155    7500.0
365    1050.0
Name: monthly_salary, dtype: float64

In [82]:
import numpy as np

#Annual salary

m1 = data['Salaire'].str.contains('par heure')
m2 = data['Salaire'].str.contains('par jour')
m3 = data['Salaire'].str.contains('par semaine')
m4 = data['Salaire'].str.contains('par mois')

data['Annual_salary'] = np.select([m1, m2, m3, m4],
                                  [data['mean_sal']*1825, #converting hourly to annual
                                   data['mean_sal']*260,   #converting daily to annual                               
                                   data['mean_sal']*52,    #weeklyly to annual
                                   data['mean_sal']*12],   #monthly to annual
                                   default=data['mean_sal']) #default value
data['Annual_salary'].head()

165    15000.0
803    18000.0
69     12000.0
299    12000.0
444    12000.0
Name: Annual_salary, dtype: float64

In [83]:
#bar graph

data.sort_values('Salaire', inplace=True)

fig = px.histogram(data, x='Annual_salary')
fig.show()

In [84]:
import regex

In [85]:
data['Adresse'] = data['Adresse'].apply(lambda x: x.split('.')[0])

In [86]:
data['Adresse'] = data['Adresse'].apply(lambda x: x.replace('\d+', '').split('.')[0])
data['Adresse']

165                                92800 puteaux
803                     92600 asnières-sur-seine
69                   hybrid remote in paris (75)
299                                  75009 paris
444                                  75009 paris
                         ...                    
986                                   paris (75)
446                                   paris (75)
437                               75002 paris 2e
155                               75002 paris 2e
365    télétravail temporaire in 75013 paris 13e
Name: Adresse, Length: 299, dtype: object

Reference: 

<a href="https://www.geeksforgeeks.org/get-all-rows-in-a-pandas-dataframe-containing-given-substring/">Get all rows in a Pandas DataFrame containing</a>

### Filtering location

In [87]:
list_com = pd.read_csv('laposte_hexasmal.csv', sep=";")
list_com.head()

,Code_commune_INSEE,Nom_commune,Code_postal,Ligne_5,Libellé_d_acheminement,coordonnees_gps
0,01009,ANDERT ET CONDON,1300,NaN,ANDERT ET CONDON,"45.7873565333,5.65788307924"
1,01012,ARANC,1110,NaN,ARANC,"46.0015344029,5.51130637511"
2,01023,ASNIERES SUR SAONE,1570,NaN,ASNIERES SUR SAONE,"46.3857281418,4.88387478623"
3,01029,BEAUPONT,1270,NaN,BEAUPONT,"46.4288360203,5.26003405676"
4,01032,BELIGNEUX,1360,NaN,BELIGNEUX,"45.8591464826,5.13989505865"


In [88]:
data['Adresse'].query(f'city=={list_com}')

AttributeError: 'Series' object has no attribute 'query'

salary

In [90]:
#Data Read, spam file, sf
import re
import nltk

# essential entity models downloads
nltk.downloader.download('maxent_ne_chunker')
nltk.downloader.download('words')
nltk.downloader.download('treebank')
nltk.downloader.download('maxent_treebank_pos_tagger')
nltk.downloader.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/tess/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/tess/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to /home/tess/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /home/tess/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/tess/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/tess/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [91]:
fsal = open(r"results.csv", "r", encoding = 'UTF8').read()

In [92]:
fsal = fsal.lower()

NLP for text processing

<a href="https://nycdatascience.com/blog/student-works/job-salary-prediction-with-python/">Job Salary Prediction with Python</a>

In [93]:
import spacy
import locationtagger
from spacy.lang.fr.examples import sentences 
from spacy.lang.en.examples import sentences

2022-02-22 09:13:45.979906: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-22 09:13:45.979931: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [94]:
spacy.prefer_gpu()
nlp = spacy.load("en_core_web_sm")

In [95]:
nlp = spacy.load("fr_core_news_sm")

In [101]:
# extracting entities.
place_entity = locationtagger.find_locations(text = data)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [97]:
place_entity.cities

['London',
 'March',
 'Vito',
 'Lucca',
 'Boston',
 'Barcelona',
 'Spring',
 'Miami',
 'Paris',
 'Meudon',
 'Montreuil',
 'Mobile',
 'Baker',
 'Jordan',
 'Belgium',
 'Belle',
 'Hartwood',
 'Swiss',
 'Leyton',
 'Spain',
 'Sec',
 'German',
 'English']

In [98]:
print(place_entity.countries)

['jordan', 'belgium', 'france', 'spain', 'switzerland']


In [99]:
print(place_entity.regions)

['Barcelona', 'Paris']


In [100]:
print(fsal.count(r"Paris"))

0


In [ ]:
import numpy as np

In [ ]:
data['Location'] = np.where(data['Adresse'] == list_com['Nom_commune'], 'True')

In [ ]:
def most_frequent(fsal):
    return max(set(fsal), key = fsal.count)

most_frequent(fsal)